The notebook preprocesses Common Voices dataset changing format to .wav with sample rate 16000 and computes labels

In [13]:
!ls 

cv-corpus-17.0-delta-2024-03-15-fr.tar.gz  en  fr  hy-AM  Preprocessing.ipynb


In [2]:
!pwd

/home/dianasimonyan/Desktop/Thesis/SpeakingRateEstimation/data/CommonVoice


In [3]:
!ls cv-corpus-17.0-2024-03-15/hy-AM/

clip_durations.tsv	 invalidated.tsv  train.tsv
clips			 other.tsv	  unvalidated_sentences.tsv
clips_wav_16khz_labeled  reported.tsv	  validated_sentences.tsv
dev.tsv			 test.tsv	  validated.tsv


In [4]:
import pandas as pd
import os
import subprocess
import tqdm

In [5]:
# Specify the path to your TSV file
file_path = 'cv-corpus-17.0-2024-03-15/hy-AM/validated.tsv'
df = pd.read_csv(file_path, sep='\t')
df.columns


Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment'],
      dtype='object')

In [6]:
data = df[['path', 'sentence']]
data.head()

,path,sentence
0,common_voice_hy-AM_31719719.mp3,Միջպետական հարաբերություններում կարևոր դեր էին...
1,common_voice_hy-AM_40085486.mp3,Նա նաև լավ էր գրում մանկական ոտանավորներ։
2,common_voice_hy-AM_39557747.mp3,Ամենամեծ ջանքերը պահանջեց աղյուսների և կղմինդր...
3,common_voice_hy-AM_39967579.mp3,"Եթե մոտակայքում չկան մեղվափեթակներ, ապա բերքատ..."
4,common_voice_hy-AM_40006367.mp3,Դպրոցը հիմնականում պահվել է եկեղեցու եկամուտնե...


In [7]:
print(data.shape)

(14813, 2)


In [8]:
data.duplicated().sum()

0

In [9]:
def AM_syl_computing(sentence):
    """The function takes armenian sentence as input and returns number of syllables in the sentence"""
    arm_vowels = ['ա', 'ե', 'է', 'ը',  'ի', 'ո', 'օ', 'ու']
    unicode_arm_vowels = [1377, 1381, 1383, 1384, 1387, 1400, 1413]
    
    sentence = sentence.lower()
    syl_count = 0
    for s in sentence:
        if ord(s) in unicode_arm_vowels:
            syl_count += 1
    return syl_count
    

In [ ]:
def Eng_syl_computing(sentence):
    vowels = ['a', 'e', 'i', 'o', 'u', 'y']
    # unicode_eng_vowels = [1377, 1381, 1383, 1384, 1387, 1400, 1413]
    sentence = sentence.lower()
    syl_count = 0
    for s in sentence:
        if s in vowels:
            syl_count += 1
    return syl_count

In [12]:
root_dir = '/home/dianasimonyan/Desktop/Thesis/SpeakingRateEstimation/data/CommonVoice/cv-corpus-17.0-2024-03-15/hy-AM/clips'
save_dir = root_dir.replace(root_dir.split(os.sep)[-1], 'clips_wav_16khz_labeled')
os.makedirs(save_dir, exist_ok=True)
sample_rate = 16000

# ffmpeg -i input.mp3 -ar 16000 output.wav
for i, row in tqdm.tqdm(data.iterrows()):
    file = row['path']
    
    if not os.path.exists(os.path.join(root_dir, file)):
        print('file does not exist')
        
    syl_count = AM_syl_computing(row['sentence'])

    proc = subprocess.Popen(['ffmpeg', '-i', f'{os.path.join(root_dir, file)}', '-ar', f'{sample_rate}', f'{os.path.join(save_dir, f"{i}_{syl_count}.wav")}'], 
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return_code = proc.wait()
        
    if return_code:
        print(f"Subprocess failed with return code {return_code}.") 
        print(f'#{i}. ', file)
        print(' '.join(['ffmpeg', '-i', f'{os.path.join(root_dir, file)}', '-ar', f'{sample_rate}', f'{os.path.join(save_dir, f"{i}_{syl_count}.wav")}']))
        print(proc.stdout)
        print(proc.stderr)
        print()
    

14813it [15:01, 16.43it/s]
